In [5]:
import tianshou
import torch
from tianshou.env.pettingzoo_env import PettingZooEnv
from pettingzoo.mpe import simple_tag_v2
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.policy import (
    BasePolicy,
    DQNPolicy,
    MultiAgentPolicyManager,
    PPOPolicy,
    RandomPolicy,
)
from tianshou.trainer import offpolicy_trainer
from tianshou.utils.net.common import Net
from utils import *
import numpy as np
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from argparse import Namespace

from causal_ccm.causal_ccm import ccm


%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
tau = 1
E = 2
env = simple_tag_v2
names = ['adversary_0', 'adversary_1', 'adversary_2', 'agent_0']


In [38]:
n = 5248
seed = 1

path = f"log/tag/ppo/board/{n=}/{seed=}"
event_acc = EventAccumulator(path)
event_acc.Reload()
random_agent = 0
assert random_agent != 4
remaining_adv = [0,1,2].remove(random_agent) 
# Show all tags in the log file
# print(event_acc.Tags())
# print(event_acc.Tensors('args/text_summary')[0])
args = eval(event_acc.Tensors('args/text_summary')[0].tensor_proto.string_val[0])
args.device = 'cpu'
epoch = 1
checkpoint_path = f'log/tag/ppo/cp/{n=}/{seed=}/{epoch=}'
args.resume_path = checkpoint_path

policy, optim, agents, _ = get_agents(env, args, override_agent=[random_agent])
print(agents)
# policy.policies[agents[1]].load_state_dict(torch.load(f"log/tag/ppo/{n}/{seed}/policy.pth"))
test_env = DummyVectorEnv([lambda: get_env(env) for i in range(10)])
replay_buffer = VectorReplayBuffer(20_000, len(test_env))
collector = Collector(policy, test_env, replay_buffer)

ep = collector.collect(n_episode=10)

data = replay_buffer.sample(0)[0]
actions = {}
for name in names:
    agent_indices = data["obs"]['agent_id'] == name
    actions[name] = data['act'][agent_indices]

L = len(actions[names[0]])

ccms = {}
for adv in remaining_adv:
    adv_name = f"adversary_{adv}"
    save_name = f'adversary_{random_agent}-on-{adv_name}'
    if save_name not in ccms.keys():
        ccms[save_name] = []
    ccm1 = ccm(actions[f'adversary_{random_agent}'], actions[adv_name], tau, E, L)
    ccms[save_name].append(ccm1.causality())


['adversary_0', 'adversary_1', 'adversary_2', 'agent_0']


In [29]:
# for i in range(1, 1001):
#     for j in range(1, 6):
#         path = f"log/tag/ppo/cp/{n=}/seed={j}/epoch={i}"
#         for f in os.listdir(path):
#             number = str(int(f.split("_")[1].split(".")[0]) - 1)
#             if "prey" in f:
#                 new_path = path+"/"+f"agent_{number}.pth"
#             elif "pred" in f:
#                 new_path = path+"/"+f"adversary_{number}.pth"

#             os.rename(path + "/" + f, new_path)

{'adversary_0': array([0, 3, 1, 3, 1, 2, 0, 0, 2, 4, 2, 2, 3, 3, 1, 1, 2, 4, 4, 4, 1, 2,
        2, 0, 4, 3, 3, 1, 0, 1, 4, 2, 2, 0, 0, 0, 3, 4, 3, 0, 1, 1, 2, 3,
        4, 1, 0, 1, 0, 4, 0, 4, 3, 0, 2, 2, 4, 0, 0, 1, 4, 3, 2, 3, 0, 2,
        3, 2, 2, 1, 0, 0, 0, 3, 1, 3, 4, 0, 1, 2, 0, 4, 1, 4, 4, 2, 1, 0,
        1, 2, 0, 3, 0, 3, 2, 0, 0, 2, 1, 4]),
 'adversary_1': array([3, 0, 3, 4, 3, 0, 4, 3, 3, 1, 0, 3, 0, 1, 0, 1, 3, 4, 3, 0, 0, 0,
        2, 0, 0, 0, 0, 2, 2, 2, 2, 3, 0, 1, 4, 0, 3, 4, 1, 3, 2, 1, 4, 2,
        1, 1, 1, 0, 0, 4, 0, 2, 3, 0, 3, 0, 3, 0, 0, 1, 3, 2, 1, 0, 0, 1,
        2, 4, 3, 2, 4, 0, 1, 1, 0, 4, 4, 3, 2, 4, 1, 0, 0, 0, 2, 0, 2, 4,
        3, 0, 3, 3, 3, 3, 2, 2, 0, 1, 3, 2]),
 'adversary_2': array([2, 0, 4, 2, 1, 1, 0, 1, 3, 4, 1, 1, 0, 1, 2, 2, 2, 3, 2, 2, 0, 1,
        4, 4, 0, 4, 4, 4, 3, 2, 2, 0, 3, 0, 1, 0, 2, 1, 3, 3, 1, 2, 3, 0,
        4, 4, 0, 2, 2, 1, 4, 2, 3, 2, 3, 1, 4, 2, 1, 1, 2, 4, 3, 0, 3, 1,
        1, 1, 4, 0, 0, 3, 0, 1, 0, 3, 0, 1, 0, 3,

In [48]:
print(ccm1.causality())
print(ccm2.causality())
print(ccm3.causality())

(0.32399378201006884, 0.0010698915633965045)
(0.31920221077058536, 0.0012802554183842904)
(0.3570675093603619, 0.0002856028800481222)
